In [10]:
using DataFrames, LowRankModels

In [11]:
data = readtable("missing_data.csv", separator = ',')
head(data)

,age,_population_in_thousands_0_4_female_,_population_in_thousands_0_4_male_,_population_in_thousands_10_14_female_,_population_in_thousands_10_14_male_,_population_in_thousands_100_female_,_population_in_thousands_100_male_,_population_in_thousands_15_19_female_,_population_in_thousands_15_19_male_,_population_in_thousands_20_24_female_,_population_in_thousands_20_24_male_,_population_in_thousands_25_29_female_,_population_in_thousands_25_29_male_,_population_in_thousands_30_34_female_,_population_in_thousands_30_34_male_,_population_in_thousands_35_39_female_,_population_in_thousands_35_39_male_,_population_in_thousands_40_44_female_,_population_in_thousands_40_44_male_,_population_in_thousands_45_49_female_,_population_in_thousands_45_49_male_,_population_in_thousands_5_9_female_,_population_in_thousands_5_9_male_,_population_in_thousands_50_54_female_,_population_in_thousands_50_54_male_,_population_in_thousands_55_59_female_,_population_in_thousands_55_59_male_,_population_in_thousands_60_64_female_,_population_in_thousands_60_64_male_,_population_in_thousands_65_69_female_,_population_in_thousands_65_69_male_,_population_in_thousands_70_74_female_,_population_in_thousands_70_74_male_,_population_in_thousands_75_79_female_,_population_in_thousands_75_79_male_,_population_in_thousands_80_84_female_,_population_in_thousands_80_84_male_,_population_in_thousands_85_89_female_,_population_in_thousands_85_89_male_,_population_in_thousands_90_94_female_,_population_in_thousands_90_94_male_,_population_in_thousands_95_99_female_,_population_in_thousands_95_99_male_,FEMALE,MALE
1,56.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
2,41.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
3,0.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
4,46.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
5,47.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
6,50.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0


In [12]:
obs = readtable("obs.csv", separator = ',', header = false)
head(obs)

,x1,x2
1,0,0
2,0,1
3,0,2
4,0,3
5,0,4
6,0,5


In [13]:
Ω = Tuple{Int64,Int64}[]
for i in 1:size(obs, 1)
    push!(Ω, (obs[i, 1] + 1, obs[i, 2] + 1))
end

3353066-element Array{Tuple{Int64,Int64},1}:
 (1, 1)     
 (1, 2)     
 (1, 3)     
 (1, 4)     
 (1, 5)     
 (1, 6)     
 (1, 7)     
 (1, 8)     
 (1, 9)     
 (1, 10)    
 (1, 11)    
 (1, 12)    
 (1, 13)    
 ⋮          
 (75973, 34)
 (75973, 35)
 (75973, 36)
 (75973, 37)
 (75973, 38)
 (75973, 39)
 (75973, 40)
 (75973, 41)
 (75973, 42)
 (75973, 43)
 (75973, 44)
 (75973, 45)

In [14]:
loss = HuberLoss()
nonneg = NonNegConstraint()
k = 10
glrm = GLRM(data, loss, nonneg, nonneg, k, obs=Ω);

# fit model
X,W,ch = fit!(glrm)

# predict Ahat
Ahat = X'*W

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /home/michael/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /home/michael/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:52
 [5] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /home/michael/.julia/v0.6/LowRankModels/src/fit.jl:18
 [6] fit!(::LowRankModels.GLRM) at /home/michael/.julia/v0.6/LowRankModels/src/fit.jl:9
 [7] include_string(::String, ::String) at ./loading.jl:515
 [8] include_string(::Module, ::String, ::String) at /home/michael/.julia/v0.6/Compat/src/Compat.jl:464
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/michael/.julia/v0.6/IJulia/src/execute_request.jl:154
 [10] eventloop(::ZMQ.Socket) at /home/michael/.jul

Fitting GLRM


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Tuple{Int64}) at ./deprecated.jl:57
 [3] Ac_mul_B at ./linalg/matmul.jl:115 [inlined]
 [4] Ac_mul_B at ./linalg/rowvector.jl:227 [inlined]
 [5] #row_objective#58(::Array{Union{Int64, Range{Int64}},1}, ::Bool, ::Function, ::LowRankModels.GLRM, ::Int64, ::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}, ::Array{Float64,2}) at /home/michael/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:29
 [6] row_objective(::LowRankModels.GLRM, ::Int64, ::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}, ::Array{Float64,2}) at /home/michael/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:27 (repeats 2 times)
 [7] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /home/michael/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:138
 [8] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /home/michael/.j

Iteration 10: objective value = 4.671860093571168e8


  likely near In[14]:7
  likely near In[14]:7
  likely near In[14]:7


Iteration 20: objective value = 1.8720806746696523e8


75973×45 Array{Float64,2}:
 26.8881   10318.0   10813.3   10374.5   …  332.986   155.227   1.08126   0.0
 26.4868   10295.6   10790.7   10351.3      332.256   154.902   1.14683   0.0
 26.4338   10306.9   10800.9   10363.2      332.631   155.075   1.11426   0.0
 28.4837   10295.8   10796.5   10355.3      332.053   154.954   1.07035   0.0
 28.0947   10314.4   10812.6   10373.6      333.318   155.233   1.14857   0.0
 26.8571   10309.9   10809.1   10369.0   …  332.823   155.159   1.07979   0.0
 27.5729   10310.6   10807.1   10368.8      332.631   155.167   1.10171   0.0
 26.8124   10311.6   10806.6   10373.3      332.358   155.169   1.06428   0.0
  3.43887   1170.9    1227.35   1177.86      37.715    17.6266  0.123878  0.0
  5.14007   1945.0    2038.94   1956.12      62.8303   29.2695  0.206708  0.0
 27.1963   10314.1   10809.5   10372.2   …  332.862   155.208   1.07946   0.0
 26.7841   10294.5   10791.5   10353.4      332.766   154.892   1.059     0.0
  5.14007   1945.0    2038.94   1956.

In [55]:
convert(DataFrame, Ahat)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95
1,6.241009342643806,-0.9742407072163678,-0.02974841600042685,-0.31882308395844766,1.987426830932081,2.1519705793502446,-0.3656027346789843,-1.016941426566626,2.0009121324099426,-1.2794437600849944,1.4502778401476972,0.618360901157633,2.1099358445827407,-0.1969247439163447,2.7998539876585884,1.821549895687747,0.915365336733712,-0.7997831767758053,-1.8328494196130394,1.0502859681271024,1.0059198540767078,1.3449999860181812,1.8453941479837035,-1.564003781626379,-1.8801363525868748,1.312221185894688,-2.41828212344978,-3.717161295826888,-3.1126424640752584,1.7703819543692745,1.618388662108435,-0.014336152635112039,0.9720143120597983,-1.0787450552688327,0.35833279727737566,-2.9169593374810434,1.1956626749934427,-2.8820852345062855,0.7709777401787085,-2.4685692600858027,-0.5834498297915183,1.3187954588089954,0.7209707766110264,-4.586690850246869,-1.8231596642824417,0.9512346507322127,0.8770116104334856,0.6444679846116345,-0.17614491125848858,1.8303138146214526,4.2943796094604645,0.7841714501162044,-1.095558967450191,0.9692586685425552,3.465138842151005,2.4840877954699536,3.4605989964670725,-3.2988908331721447,0.9046164531544836,-0.08458132366187099,0.0018257812755946088,-4.209401621791027,-1.9600988390564615,3.486730082045989,-1.0290155977724587,0.46130285770717144,-0.06263319492154742,-2.552280240855593,-0.17499922425849992,-0.7527055132978129,1.112611266810871,-0.5462540822174229,-1.111080127204066,-0.013470171673792102,-2.036008352718265,4.651502896321507,0.39343965542716736,-0.1290340767140191,1.3554678595020415,-3.482737124300948,2.1082993760449256,0.5383659815776421,-2.0355476333510794,-0.4014185743487108,-1.505416131951795,0.246842584752017,0.07480927473835215,-1.40647223447554,-0.9581907175179593,0.5044669950153556,1.3880293463878859,0.4440824892515934,3.1388309057244053,0.8800134078020109,0.7077608661722787
2,0.2777717118517827,-0.2008056569524806,-0.07621061317154897,0.03731788380004099,0.009408356081761807,-0.1936601496488864,0.045051936805589894,0.030850257417067886,0.22983383678938163,0.04938772747453744,0.13944261760380788,-0.027628321054198976,0.26569846487306636,-0.09968752947351246,0.049074670693944444,0.0845907029564708,-0.10724016057301,0.18117048618387058,0.003194430332461442,0.18739658866266543,0.2585987332649236,0.1534909848719477,0.035841421198165496,-0.07310323010108903,-0.027066188967953115,0.19068168194641694,-0.039907773001586454,-0.24405404510163958,0.04578983476771135,0.07965851733950255,-0.06816901485279962,0.04413902112194405,-0.017779424561266553,-0.09544981023933106,-0.09311740276587366,-0.1929737552398659,0.18150304329130457,-0.027715998394389495,-0.013401228095845047,-0.22475606827012798,0.1414469615319161,0.02354475782825996,-0.11079922360376052,-0.19509069729510248,-0.014889510716298633,0.110242064656767,0.3272965425699017,0.09153146810670819,-0.05052750439380788,0.07187667367295639,0.37930154935967525,0.07212273766700632,-0.13201576561457196,0.17797501458528817,-0.0012152145312841578,0.10800355747251524,0.18613027754439265,-0.17977411532121704,0.1814735885048838,-0.048459634194228336,-0.25467575885799054,-0.18878219592869858,-0.08851494878009378,0.23706060915275615,-0.16108883950452438,0.21794883276344568,0.14978920018260447,-0.053360638669616385,0.008820864065212132,0.044404769345667665,-0.02188029260730007,0.026795165075216607,-0.008761778412075111,-0.0398336298049255,-0.20213605180319083,0.20292095410479027,0.14452420314057046,-0.11009528018907955,0.04552439461512691,-0.055227741868052925,0.250887161337406,0.03756244566179705,-0.10424404221001314,-0.017981762627450746,-0.21746932064221414,0.17327271086027607,-0.198230318535192,-